# Climate Change Modeling

In [ ]:
# 03 — Projections & Scenarios

# --- Hybrid dataset loader ---
import os, io
import pandas as pd

DATA_PATH = "../data/future_climate_data.csv"

if os.path.exists(DATA_PATH):
    print(f"Loading dataset from {DATA_PATH}")
    df = pd.read_csv(DATA_PATH)
else:
    try:
        from google.colab import files
        uploaded = files.upload()
        file_name = list(uploaded.keys())[0]
        df = pd.read_csv(io.BytesIO(uploaded[file_name]))
        print(f"Loaded uploaded file: {file_name}")
    except Exception as e:
        from tkinter import Tk, filedialog
        Tk().withdraw()
        file_path = filedialog.askopenfilename(filetypes=[("CSV files","*.csv")])
        df = pd.read_csv(file_path)
        print(f"Loaded local file: {file_path}")

# --- Load trained model ---
import pickle
from sklearn.preprocessing import StandardScaler

ART_DIR = "../artifacts"
MODEL_PTH = os.path.join(ART_DIR, "rf_model.pkl")
SCALER_PTH = os.path.join(ART_DIR, "rf_scaler.pkl")

with open(MODEL_PTH, "rb") as f:
    model = pickle.load(f)
with open(SCALER_PTH, "rb") as f:
    scaler: StandardScaler = pickle.load(f)

# Ensure numeric features used in training are present
num_cols = df.select_dtypes("number").columns.tolist()
X_future = scaler.transform(df[num_cols])
preds = model.predict(X_future)
df["prediction"] = preds

print(df.head())

In [ ]:
# Save projections
out_path = "../reports/projections.csv"
df.to_csv(out_path, index=False)
print("Saved:", out_path)